# Extract features, retrain Sherlock and generate predictions.

### Necessary imports

In [2]:
import pandas as pd
import numpy as np
import sys
import pickle
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.metrics import f1_score
sys.path.append("..")

In [3]:
from src.features.build_features import build_features
from src.deploy.train_sherlock import train_sherlock
from src.deploy.predict_sherlock import predict_sherlock

C:\Users\hulse504\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\hulse504\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


### Load small raw data sample and corresponding labels

In [4]:
data = pd.read_csv('../data/raw/test_values.csv', sep=',', index_col=0, header=None)
labs = pd.read_csv('../data/raw/test_labels.csv', sep=',', index_col=0, header=None)

In [5]:
data.head()

,1
0,
20368,"['Central Missouri', 'unattached', 'unattached..."
664102,"[95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, ..."
366813,"['Katie Crews', 'Christian Hiraldo', 'Alex Est..."
530567,"['Christian', 'Non-Christian', 'Unreported', '..."
176253,"['AAF-McQuay Canada Inc.', 'AAF-McQuay Canada ..."


In [27]:
data = data.iloc[:,0].apply(literal_eval)

In [27]:
data.head()

0
20368     [Central Missouri, unattached, unattached, Kan...
664102    [95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, ...
366813    [Katie Crews, Christian Hiraldo, Alex Estrada,...
530567    [Christian, Non-Christian, Unreported, Jewish,...
176253    [AAF-McQuay Canada Inc., AAF-McQuay Canada Inc...
Name: 1, dtype: object

In [24]:
labs.head()

,1
0,
20368,affiliation
664102,weight
366813,jockey
530567,religion
176253,company


### Extract features of raw data samples from dataframes and save preprocessed data

In [28]:
data.values[0]

['Central Missouri',
 'unattached',
 'unattached',
 'Kansas State University',
 'unattached',
 'North Dakota State',
 'Nike']

In [60]:
X_train = build_features(data.values)
y_train = labs.values.flatten()
print('Extracted features.')

In [56]:
with open('../data/processed/X_train.data', 'wb') as f:
    pickle.dump(X_train, f)
    
with open('../data/processed/y_train.data', 'wb') as f:
    pickle.dump(y_train, f)

In [57]:
with open('../data/processed/X_train.data', 'rb') as f:
    X_train = pickle.load(f)
    
with open('../data/processed/y_train.data', 'rb') as f:
    y_train = pickle.load(f)

### Train sherlock on new data

In [53]:
# For simplicity provide X_train as validation set.
train_sherlock(X_train, y_train, X_train, y_train, 'retrain_minimal_sample') 
print('Trained new model.')

Successfully loaded and compiled model, now fitting.
Train on 1001 samples, validate on 1001 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1001/1001 [==============================] - 3s 3ms/sample - loss: 4.8743 - categorical_accuracy: 0.0130 - val_loss: 4.8196 - val_categorical_accuracy: 0.0180
Epoch 2/100
1001/1001 [==============================] - 1s 717us/sample - loss: 4.8063 - categorical_accuracy: 0.0140 - val_loss: 4.6911 - val_categorical_accuracy: 0.0210
Epoch 3/100
1001/1001 [==============================] - 1s 587us/sample - loss: 4.6949 - categorical_accuracy: 0.0310 - val_loss: 4.6318 - val_categorical_accuracy: 0.0210
Epoch 4/100
1001/1001 [==============================] - 1s 593us/sample - loss: 4.6241 - categorical_accuracy: 0.0220 - val_loss: 4.5949 - val_categorical_accuracy: 0.0250
Epoch 5/100
1001/1001 [==============================] - 1s 588us/sample - loss: 4.5290 - categorical_accuracy: 0.0420 - val_loss: 4.5686 - val_categorical_accur

Epoch 46/100
1001/1001 [==============================] - 1s 738us/sample - loss: 2.5997 - categorical_accuracy: 0.4875 - val_loss: 3.3346 - val_categorical_accuracy: 0.4486
Epoch 47/100
1001/1001 [==============================] - 1s 901us/sample - loss: 2.5520 - categorical_accuracy: 0.4925 - val_loss: 3.2833 - val_categorical_accuracy: 0.4595
Epoch 48/100
1001/1001 [==============================] - 1s 856us/sample - loss: 2.5012 - categorical_accuracy: 0.5295 - val_loss: 3.2319 - val_categorical_accuracy: 0.4755
Epoch 49/100
1001/1001 [==============================] - 1s 759us/sample - loss: 2.4333 - categorical_accuracy: 0.5395 - val_loss: 3.1805 - val_categorical_accuracy: 0.4835
Epoch 50/100
1001/1001 [==============================] - 1s 735us/sample - loss: 2.3844 - categorical_accuracy: 0.5445 - val_loss: 3.1280 - val_categorical_accuracy: 0.4935
Epoch 51/100
1001/1001 [==============================] - 1s 669us/sample - loss: 2.3656 - categorical_accuracy: 0.5455 - val_loss

### Generate predictions with the retrained model

In [58]:
# Predict labels using the retrained model (with nn_id retrain_minimal_sample)
predicted_labels = predict_sherlock(X_train, 'retrain_minimal_sample')
print('Predicted labels: ', predicted_labels, 'true labels: ', y_train)

Predicted labels:  ['team' 'weight' 'jockey' ... 'company' 'plays' 'jockey'] true labels:  ['affiliation' 'weight' 'jockey' ... 'company' 'plays' 'continent']


In [59]:
f1_score(y_train, predicted_labels, average='weighted')

0.8086928777494169